In [1]:
from pysdql import DataFrame
import pysdql as pd
from pysdql.core.dtypes.sdql_ir import (
    ConstantExpr,
    PrintAST,
    GenerateSDQLCode,
)

# Q1

In [2]:
li = DataFrame()

li_filt = li[(li.l_shipdate <= "1998-09-02")]
li_filt["disc_price"] = li_filt.l_extendedprice * (1 - li_filt.l_discount)
li_filt["charge"] = li_filt.l_extendedprice * (1 - li_filt.l_discount) * (1 + li_filt.l_tax)

result = li_filt \
    .groupby(["l_returnflag", "l_linestatus"]) \
    .agg(sum_qty=("l_quantity", "sum"),
         sum_base_price=("l_extendedprice", "sum"),
         sum_disc_price=("disc_price", "sum"),
         sum_charge=("charge", "sum"),
         count_order=("l_quantity", "count")
         )

In [3]:
print(result.operations)

{'iter': False, 'op': CompareExpr(CompareSymbol.LTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_shipdate'), ConstantExpr(19980902)), 'op_type': <class 'pysdql.core.dtypes.CondExpr.CondExpr'>, 'on': li, 'ret_type': None}
{'iter': False, 'op': {'disc_price': MulExpr(RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice'), SubExpr(ConstantExpr(1), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount')))}, 'op_type': <class 'pysdql.core.dtypes.VirColEl.VirColEl'>, 'on': li, 'ret_type': None}
{'iter': False, 'op': {'charge': MulExpr(MulExpr(RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice'), SubExpr(ConstantExpr(1), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount'))), AddExpr(ConstantExpr(1), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_tax')))}, 'op_type': <class 'pysdql.core.dtypes.VirColEl.VirColEl'>, 'on': li, 'ret_type': None}
{'iter': True, 'op': {'sum_qty': RecAccessExpr(PairAccessExpr(x_li, 0), 'l_quantity'), 'sum_base_price': RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extended

In [4]:
print(result.optimize())

LetExpr(li_groupby_agg, SumExpr(x_li, db->li_dataset, IfExpr(CompareExpr(CompareSymbol.LTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_shipdate'), ConstantExpr(19980902)), DicConsExpr([(RecConsExpr([('l_returnflag', RecAccessExpr(PairAccessExpr(x_li, 0), 'l_returnflag')), ('l_linestatus', RecAccessExpr(PairAccessExpr(x_li, 0), 'l_linestatus'))]), RecConsExpr([('sum_qty', RecAccessExpr(PairAccessExpr(x_li, 0), 'l_quantity')), ('sum_base_price', RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice')), ('sum_disc_price', MulExpr(RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice'), SubExpr(ConstantExpr(1), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount')))), ('sum_charge', MulExpr(MulExpr(RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice'), SubExpr(ConstantExpr(1), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount'))), AddExpr(ConstantExpr(1), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_tax')))), ('count_order', ConstantExpr(1))]))]), ConstantExpr(None)), False), LetExpr(l

In [5]:
PrintAST(result.optimize())

nodeType: <class 'pysdql.core.dtypes.sdql_ir.LetExpr'> | ID: 140
├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 95 | name: li_groupby_agg
├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.SumExpr'> | ID: 124
│      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 1 | name: x_li
│      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 2 | name: db->li_dataset
│      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.IfExpr'> | ID: 122
│      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.CompareExpr'> | ID: 113 | compareType: CompareSymbol.LTE
│      │      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.RecAccessExpr'> | ID: 4 | name: l_shipdate
│      │      │      │      └───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.PairAccessExpr'> | ID: 3 | index: 0
│      │      │      │             └───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 1 | name: x_li
│      │      │      └───── 

In [6]:
print(GenerateSDQLCode(result.optimize()))

let li_groupby_agg = sum (x_li in db->li_dataset) if (x_li[0].l_shipdate <= 19980902) then { < l_returnflag: x_li[0].l_returnflag, l_linestatus: x_li[0].l_linestatus> -> < sum_qty: x_li[0].l_quantity, sum_base_price: x_li[0].l_extendedprice, sum_disc_price: x_li[0].l_extendedprice * 1 - x_li[0].l_discount, sum_charge: x_li[0].l_extendedprice * 1 - x_li[0].l_discount * 1 + x_li[0].l_tax, count_order: 1> } else none in let li_filter_insert_insert_groupby_agg = sum (v8 in li_groupby_agg) { recordconcat(v8[0],v8[1]) -> true } in let out = li_filter_insert_insert_groupby_agg in true


# Q3

In [7]:
cu = DataFrame()
ord = DataFrame()
li = DataFrame()

cu_filt = cu[cu.c_mktsegment == "BUILDING"]
cu_filt = cu_filt[["c_custkey"]]

# print(cu_filt.operations)

ord_filt = ord[ord.o_orderdate < "1995-03-15"]
ord_cu_join = pd.merge(cu_filt, ord_filt, left_on="c_custkey", right_on="o_custkey", how="inner")
ord_cu_join = ord_cu_join[["o_orderkey", "o_orderdate", "o_shippriority"]]

# li_filt = li[li.l_shipdate > "1995-03-15"]
# li_order_join = pd.merge(ord_cu_join, li_filt, left_on="o_orderkey", right_on="l_orderkey", how="inner")
# li_order_join["revenue"] = li_order_join.l_extendedprice * (1 - li_order_join.l_discount)
#
# result = li_order_join \
#     .groupby(["l_orderkey", "o_orderdate", "o_shippriority"]) \
#     .agg(revenue=("revenue", "sum"))

# return result

In [8]:
print(cu_filt.operations)

{'iter': False, 'op': CompareExpr(CompareSymbol.EQ, RecAccessExpr(PairAccessExpr(x_cu, 0), 'c_mktsegment'), ConstantExpr("BUILDING")), 'op_type': <class 'pysdql.core.dtypes.CondExpr.CondExpr'>, 'on': cu, 'ret_type': None}
{'iter': False, 'op': {'proj_on': cu, 'proj_cols': ['c_custkey']}, 'op_type': <class 'pysdql.core.dtypes.ColProjExpr.ColProjExpr'>, 'on': cu, 'ret_type': None}
{'iter': True, 'op': {'left': cu, 'right': ord, 'how': 'inner', 'left_on': 'c_custkey', 'right_on': 'o_custkey'}, 'op_type': <class 'pysdql.core.dtypes.MergeExpr.MergeExpr'>, 'on': cu, 'ret_type': None}


In [9]:
print(ord_cu_join.operations)

{'iter': False, 'op': CompareExpr(CompareSymbol.LT, RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_orderdate'), ConstantExpr(19950315)), 'op_type': <class 'pysdql.core.dtypes.CondExpr.CondExpr'>, 'on': ord, 'ret_type': None}
{'iter': True, 'op': {'left': cu, 'right': ord, 'how': 'inner', 'left_on': 'c_custkey', 'right_on': 'o_custkey'}, 'op_type': <class 'pysdql.core.dtypes.MergeExpr.MergeExpr'>, 'on': ord, 'ret_type': None}
{'iter': False, 'op': {'proj_on': ord, 'proj_cols': ['o_orderkey', 'o_orderdate', 'o_shippriority']}, 'op_type': <class 'pysdql.core.dtypes.ColProjExpr.ColProjExpr'>, 'on': ord, 'ret_type': None}


In [10]:
print(cu_filt.merge_left_stmt(ConstantExpr('Here is the right part of hash join!')))

LetExpr(cu_filter_proj_merge, SumExpr(x_cu, db->cu_dataset, IfExpr(CompareExpr(CompareSymbol.EQ, RecAccessExpr(PairAccessExpr(x_cu, 0), 'c_mktsegment'), ConstantExpr("BUILDING")), DicConsExpr([(RecConsExpr([('c_custkey', RecAccessExpr(PairAccessExpr(x_cu, 0), 'c_custkey'))]), RecConsExpr([('c_custkey', RecAccessExpr(PairAccessExpr(x_cu, 0), 'c_custkey'))]))]), ConstantExpr(None)), False), ConstantExpr("Here is the right part of hash join!"))


In [11]:
print(ord_cu_join.merge_right_stmt(ConstantExpr('Here is the next LetExpr!')))

LetExpr(ord_filter_merge_proj, SumExpr(x_ord, db->ord_dataset, IfExpr(CompareExpr(CompareSymbol.LT, RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_orderdate'), ConstantExpr(19950315)), IfExpr(CompareExpr(CompareSymbol.NE, DicLookupExpr(cu_filter_proj_merge, RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_custkey')), ConstantExpr(None)), DicConsExpr([(RecConsExpr([('o_custkey', RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_custkey'))]), RecConsExpr([('o_orderkey', RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_orderkey')), ('o_orderdate', RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_orderdate')), ('o_shippriority', RecAccessExpr(PairAccessExpr(x_ord, 0), 'o_shippriority'))]))]), ConstantExpr(None)), ConstantExpr(None)), False), ConstantExpr("Here is the next LetExpr!"))


# Q6

In [12]:
li = DataFrame()

li_filt = li[
    (li.l_shipdate >= "1994-01-01") &
    (li.l_shipdate < "1995-01-01") &
    (li.l_discount >= 0.05) &
    (li.l_discount <= 0.07) &
    (li.l_quantity < 24)
    ]

result = (li_filt.l_extendedprice * li_filt.l_discount).sum()

In [13]:
print(result.operations)

{'iter': False, 'op': MulExpr(MulExpr(MulExpr(MulExpr(CompareExpr(CompareSymbol.GTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_shipdate'), ConstantExpr(19940101)), CompareExpr(CompareSymbol.LT, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_shipdate'), ConstantExpr(19950101))), CompareExpr(CompareSymbol.GTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount'), ConstantExpr(0.05))), CompareExpr(CompareSymbol.LTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount'), ConstantExpr(0.07))), CompareExpr(CompareSymbol.LT, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_quantity'), ConstantExpr(24))), 'op_type': <class 'pysdql.core.dtypes.CondExpr.CondExpr'>, 'on': li, 'ret_type': None}
{'iter': True, 'op': MulExpr(RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice'), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount')), 'op_type': <class 'pysdql.core.dtypes.AggrExpr.AggrExpr'>, 'on': li, 'ret_type': None}


In [14]:
print(result.optimize())

LetExpr(result, SumExpr(x_li, db->li_dataset, IfExpr(MulExpr(MulExpr(MulExpr(MulExpr(CompareExpr(CompareSymbol.GTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_shipdate'), ConstantExpr(19940101)), CompareExpr(CompareSymbol.LT, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_shipdate'), ConstantExpr(19950101))), CompareExpr(CompareSymbol.GTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount'), ConstantExpr(0.05))), CompareExpr(CompareSymbol.LTE, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount'), ConstantExpr(0.07))), CompareExpr(CompareSymbol.LT, RecAccessExpr(PairAccessExpr(x_li, 0), 'l_quantity'), ConstantExpr(24))), MulExpr(RecAccessExpr(PairAccessExpr(x_li, 0), 'l_extendedprice'), RecAccessExpr(PairAccessExpr(x_li, 0), 'l_discount')), ConstantExpr(0.0)), False), LetExpr(out, result, ConstantExpr(True)))


In [15]:
PrintAST(result.optimize())

nodeType: <class 'pysdql.core.dtypes.sdql_ir.LetExpr'> | ID: 432
├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 425 | name: result
├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.SumExpr'> | ID: 426
│      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 330 | name: x_li
│      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.VarExpr'> | ID: 331 | name: db->li_dataset
│      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.IfExpr'> | ID: 423
│      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.MulExpr'> | ID: 422
│      │      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.MulExpr'> | ID: 350
│      │      │      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.MulExpr'> | ID: 345
│      │      │      │      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.MulExpr'> | ID: 340
│      │      │      │      │      │      ├───── nodeType: <class 'pysdql.core.dtypes.sdql_ir.CompareExpr'> | ID: 338 

In [16]:
print(GenerateSDQLCode(result.optimize()))

let result = sum (x_li in db->li_dataset) if (x_li[0].l_shipdate >= 19940101 * x_li[0].l_shipdate < 19950101 * x_li[0].l_discount >= 0.05 * x_li[0].l_discount <= 0.07 * x_li[0].l_quantity < 24) then x_li[0].l_extendedprice * x_li[0].l_discount else 0.0 in let out = result in true
